In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
%pip install pygame

Indepndencies

In [1]:
import gymnasium as gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_atari_env
import pygame
import os


In [ ]:
!python -m atari_py.import_roms .\ROMS

Test

In [2]:
environment_name='Breakout-v4'
env=gym.make(environment_name,render_mode='human') 

In [3]:
env.reset()

(array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 {'lives': 5, 'episode_frame_number': 0, 'frame_number': 0})

In [4]:
env.action_space

Discrete(4)

In [5]:
env.observation_space

Box(0, 255, (210, 160, 3), uint8)

In [6]:
episodes=5
for episode in range (1,episodes+1):
    obs=env.reset()
    done=False
    score=0
    while not done:
        env.render()
        action=env.action_space.sample()
        obs, reward, done, any, info = env.step(action)
        score += reward
    print('Episode:{} Score{}'.format(episode,score))
pygame.quit()
env.close()        

e:\Reinforcment-Learning\AtariRLModel\.conda\Lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Episode:1 Score0.0
Episode:2 Score2.0
Episode:3 Score3.0
Episode:4 Score4.0
Episode:5 Score0.0


Vectorize Env and Train Model

In [7]:
env=make_atari_env('Breakout-v4',n_envs=4,seed=0)
env=VecFrameStack(env,n_stack=4)

In [8]:
log_path=os.path.join('Training','Logs')
model=A2C('CnnPolicy', env, verbose=1,tensorboard_log=log_path)

e:\Reinforcment-Learning\AtariRLModel\.conda\Lib\site-packages\torch\cuda\__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ..\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Using cpu device
Wrapping the env in a VecTransposeImage.


In [9]:
model.learn(total_timesteps=100000)

Logging to Training\Logs\A2C_1
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 267      |
|    ep_rew_mean        | 1.32     |
| time/                 |          |
|    fps                | 144      |
|    iterations         | 100      |
|    time_elapsed       | 13       |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -1.38    |
|    explained_variance | 0.0714   |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 0.341    |
|    value_loss         | 0.245    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 281      |
|    ep_rew_mean        | 1.58     |
| time/                 |          |
|    fps                | 151      |
|    iterations         | 200      |
|    time_elapsed       | 26       |
|    total_timesteps    | 4000     |
| train

Save and Reload

In [10]:
a2c_path=os.path.join('Training','Saved Models','A2C_Atari_Model')
model.save(a2c_path)

In [11]:
del model

In [12]:
model=A2C.load(a2c_path,env)

Wrapping the env in a VecTransposeImage.


Evaluate(can only be on 1 env can't be vectorized) and Test

In [13]:
env=make_atari_env('Breakout-v4',n_envs=1,seed=0)
env=VecFrameStack(env,n_stack=4)

In [14]:

evaluate_policy(model=model,env=env,n_eval_episodes=10, render=True)


e:\Reinforcment-Learning\AtariRLModel\.conda\Lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


(7.9, 2.467792535850613)